In [1]:
import numpy as np 
import os
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.models import *
from keras.layers import *
from keras import initializers
from keras import backend as K
import tensorflow as tf
import numpy as np 
import os
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

from keras.losses import categorical_crossentropy

import tensorflow as tf

import sys
    
sys.path.insert(1,r"C:\Users\adame\Desktop\Inzynierka")

In [2]:
from generator import image_load_generator_x,image_load_generator_mask,image_load_generator_mask_binary,image_load_generator_mask_nocat


In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="Model-Dice--1channels-new", entity="adamsoja")




C:\anaconda3\envs\machine\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "
wandb: Currently logged in as: adamsoja. Use `wandb login --relogin` to force relogin


In [3]:
batch_size = 10
train_X = image_load_generator_x('../brains/train',batch_size)
train_mask_gen = image_load_generator_mask('../brains/train',batch_size)

val_X = image_load_generator_x('../brains/valid',batch_size)
val_mask_gen = image_load_generator_mask('../brains/valid',batch_size)
import os

steps_train = len(os.listdir('../brains/train/mask')) // batch_size
steps_val = len(os.listdir('../brains/valid/mask')) // batch_size



In [4]:
train_datagen = zip(train_X,train_mask_gen)
val_datagen = zip(val_X,val_mask_gen)

In [5]:
def tversky(y_true, y_pred):
    smooth=1.
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)


#posprawdzać

In [6]:
def dice_coef(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice



def dice_coef_multilabel(y_true, y_pred):
    dice=0
    for index in range(4):
        dice += dice_coef(y_true[:,:,:,index], y_pred[:,:,:,index])
    return (1 - dice/4)


loss_cross = tf.keras.losses.CategoricalCrossentropy()

def dice_coef_loss(y_true, y_pred, smooth=1.):
    return ((1 - dice_coef(y_true, y_pred,1.)) + loss_cross(y_true,y_pred))

metrics = [dice_coef,tf.keras.metrics.MeanIoU(num_classes=4),'accuracy',tf.keras.metrics.CategoricalAccuracy()]


#wagi class

In [7]:

def multi_unet_model(n_classes=4, IMG_HEIGHT=240, IMG_WIDTH=240, IMG_CHANNELS=1):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.1)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)

    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    

    return model


import keras
opt = keras.optimizers.Adam(learning_rate=0.001)

model = multi_unet_model(n_classes=4, IMG_HEIGHT=240, IMG_WIDTH=240, IMG_CHANNELS=1)

model.compile(optimizer=opt, loss=dice_coef_multilabel, metrics=[metrics])

#model.summary()
class_weights = [0.3,1,1,1]

In [8]:
model.fit(train_datagen,
                    steps_per_epoch = steps_train,
                    verbose=1,
                    epochs=1, 
                    validation_data=val_datagen, 
                    validation_steps = steps_val,
                    shuffle=False,
                    #callbacks=[WandbCallback()],
                   )
                    
    

C:\anaconda3\envs\machine\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


3079/3079 [==============================] - 407s 128ms/step - loss: 0.6761 - dice_coef: 0.9737 - mean_io_u: 0.9660 - accuracy: 0.9742 - categorical_accuracy: 0.9742 - val_loss: 0.6101 - val_dice_coef: 0.9740 - val_mean_io_u: 0.9660 - val_accuracy: 0.9740 - val_categorical_accuracy: 0.9740


In [9]:
model.save('../models/dice_coef__1chan.h5')

In [ ]:
def make_accuracy_plot(history):
    """
    Accuracy plot of model 
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set()
    acc, val_acc = history.history['dice_coef'], history.history['val_dice_coef']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(10, 8))
    plt.plot(epochs, acc, label='Training dice', marker='o')
    plt.plot(epochs, val_acc, label='Validation dice', marker='o')
    plt.legend()
    plt.title('Dokładność trenowania i walidacji')
    plt.xlabel('Epoki')
    plt.ylabel('Dice')
    plt.show()

def make_loss_plot(history):
    """
    Loss plot of model
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set()
    loss, val_loss = history.history['loss'], history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    
    plt.figure(figsize=(10, 8))
    plt.plot(epochs, loss, label='Training loss', marker='o')
    plt.plot(epochs, val_loss, label='Validation loss', marker='o')
    plt.legend()
    plt.title('Strata trenowania i walidacji')
    plt.xlabel('Epoki')
    plt.ylabel('Strata')
    plt.show()


In [ ]:
make_accuracy_plot(history)

In [ ]:
make_loss_plot(history)

In [1]:
import sys
    
sys.path.insert(1,r"C:\Users\adame\Desktop\Inzynierka")

In [2]:
import tensorflow as  tf

In [3]:
import numpy as np 
from generator import image_load_generator_x,image_load_generator_mask,image_load_generator_mask_nocat
import keras
import matplotlib.pyplot as plt
import random
import tensorflow as  tf
test_generator = image_load_generator_x('brains/test',50)
mask_generator =image_load_generator_mask_nocat('brains/test',50)
model = tf.keras.models.load_model('../models/dice_coef__1chan.h5',compile=False)

In [ ]:
X_test = next(test_generator)
Y_test = next(mask_generator)
y_pred = model.predict(X_test)


predicted_img=np.argmax(y_pred,axis=3)
number = random.randint(0,49)

brain = X_test[number]
plt.figure(figsize=(30, 20))
plt.subplot(241)
plt.title('Zdjecie mozgu')
plt.imshow(brain,cmap='gray')
plt.subplot(242)
plt.title('Segmentacja przez eksperta')
plt.imshow(Y_test[number],cmap='gray')
plt.subplot(243)
plt.title('Segmentacja przez sieć')
plt.imshow(predicted_img[number],cmap='gray')

plt.show()

In [14]:
y_pred.shape

(50, 240, 240, 4)

In [46]:
np.unique(predicted_img)

array([0], dtype=int64)

In [ ]:
table = wandb.Table(columns=['ID', 'Image'])

train_X2 = image_load_generator_x('../brains/train',batch_size)
train_mask_gen2 = image_load_generator_mask_nocat('../brains/train',batch_size)


class_labels = {
  0: "background",
  1: "necrotic and non-enhancing tumor",
  2: "edema",
    4: 'enhancing tumor'
}


images = next(train_X2)
labels = next(train_mask_gen2)

for i in range(batch_size):
    img = images[i]
    
    wandb.log(
  {"my_image_key" : wandb.Image(img[:,:,2], masks={
    "predictions" : {
        "mask_data" : labels[i],
        "class_labels" : class_labels
    },
    "ground_truth" : {
        "mask_data" : labels[i],
        "class_labels" : class_labels
    }
})})



In [ ]:
                flipped = tf.image.flip_left_right(X_train)
                x_train.append(flipped)
                
                rotated = tf.image.rot90(X_train)
                x_train.append(rotated)
                
                rotated2 = tf.image.rot90(rotated)
                x_train.append(rotated2)
                
                
                flipped = tf.image.flip_left_right(X_train)
                x_train.append(flipped)
                
                rotated = tf.image.rot90(X_train)
                x_train.append(rotated)
                
                rotated2 = tf.image.rot90(rotated)
                x_train.append(rotated2)
                